# AIB DataHack

Ok let's give this machine learning malarkey a go...

## Data Import and Initial Prep

In [6]:
# library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [7]:
import keras
keras.__version__

ModuleNotFoundError: No module named 'keras'

In [3]:
# data import
df = pd.read_csv('data/training_small_processed.csv')
df.shape

(4999, 82)

In [4]:
df.head()

,BuildingID,Year,EnergyRatingCat,EnergyRatingCont,GroundFloorArea,AvgWallU,AvgRoofU,AvgFloorU,AvgWindowU,AvgDoorU,...,ProdFirstEnerDelivered,ProdFirstEnerConvFac,ProdSecondEnerDelivered,ProdSecondEnerConvFac,ProdThirdEnerDelivered,ProdThirdEnerConvFac,PrimaryEnergySecondarySpace,EnergyMainWater,MainSHFuel_cat,MainWHFuel_cat
0,190835,1890,F,418.42,52.04,2.10,2.30,0.61,2.71,3.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,2335.6809,3,4
1,523113,1901,G,520.35,147.44,2.01,0.75,0.94,2.63,3.960000,...,0.0,0.0,0.0,0.0,0.0,0.0,7144.838,4470.0759,3,4
2,271393,1968,D2,281.33,104.84,0.38,0.13,0.68,2.81,2.482515,...,0.0,0.0,0.0,0.0,0.0,0.0,2127.347,5837.4729,3,4
3,585961,2002,C2,188.17,95.31,0.55,0.26,0.34,2.80,3.030000,...,0.0,0.0,0.0,0.0,0.0,0.0,2771.886,3769.6248,5,6
4,624083,2002,B3,134.95,283.87,0.55,0.26,0.41,3.10,3.010000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,5296.5828,5,6


In [28]:
target = train['EnergyRatingCat']
del train['EnergyRatingCat']
del train['EnergyRatingCont']
del train['MainSHFuel']
del train['MainWHFuel']
target.head()

0    F 
1    G 
2    D2
3    C2
4    B3
Name: EnergyRatingCat, dtype: object

## First Model

Ok first, lets split the data into training and testing sets. For this initial attempt, I'm going to just use the U values to train the model: AvgWallU, AvgRoofU,AvgFloorU, AvgWindowU and AvgDoorU.

In [29]:
cols = ['AvgWallU', 'AvgRoofU','AvgFloorU', 'AvgWindowU', 'AvgDoorU']
X = train[cols]

In [30]:
# Quick check to make sure there's no null values in any of the columns
X.isnull().sum()

AvgWallU      0
AvgRoofU      0
AvgFloorU     0
AvgWindowU    0
AvgDoorU      0
dtype: int64

In [34]:
# Convert string categories to numerical representation
y = target.copy().astype('category').cat.codes

In [35]:
# train/test split
train_X, test_X, train_y, test_y = train_test_split(X, y)

In [36]:
train_X.shape

(3749, 5)

In [38]:
my_model = RandomForestClassifier()
my_model.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [43]:
y_pred = my_model.predict(test_X)

In [44]:
# heads up accuracy
accuracy_score(test_y, y_pred)

0.2384

In [51]:
# F1 Score
print('Weighted: ', f1_score(test_y, y_pred, average='weighted'))
print('Macro: ', f1_score(test_y, y_pred, average='macro'))
print('Micro: ', f1_score(test_y, y_pred, average='micro'))

Weighted:  0.233533188098
Macro:  0.215401575134
Micro:  0.2384


In [74]:
# confusion matrix
cm = confusion_matrix(test_y, y_pred)
cm.diagonal().sum()/cm.sum()
cm

array([[ 0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1, 30,  1,  3,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  5,  1,  6,  9,  1,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  6,  3, 10, 10,  5,  2,  2,  2,  1,  1,  0,  0,  0],
       [ 0,  0,  2,  2,  8, 27, 23, 12,  9,  8,  3,  2,  0,  0,  0],
       [ 0,  0,  1,  1,  6, 20, 24, 17, 14, 15,  8,  2,  2,  0,  2],
       [ 0,  0,  0,  0,  5, 14, 27, 40, 26, 21, 16,  2,  1,  3,  1],
       [ 0,  0,  0,  0,  5, 11, 17, 35, 39, 31, 22,  3,  3,  2,  0],
       [ 0,  0,  0,  0,  2, 12, 15, 25, 26, 35, 33, 15,  6,  3,  2],
       [ 0,  0,  0,  0,  2,  7, 14, 18, 24, 27, 24,  7,  5,  6,  3],
       [ 0,  0,  0,  0,  0,  2,  7,  8,  4, 15, 19, 11,  4,  7,  7],
       [ 0,  0,  0,  0,  0,  2,  2,  6,  5,  9, 17,  8,  4,  6,  9],
       [ 0,  0,  0,  0,  0,  0,  3,  2,  5,  9,  6,  2,  4, 14, 17],
       [ 0,  0,  0,  0,  0,  1,  1

## Second Model

Lets use the exact same model except create dummy variables for the target labels instead of encoding them numerically.

In [75]:
y = pd.get_dummies(target)
y.head()

,A1,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,E1,E2,F,G
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [76]:
train_X, test_X, train_y, test_y = train_test_split(X, y)
train_X.shape

(3749, 5)

In [77]:
train_y.shape

(3749, 15)

In [78]:
my_model2 = RandomForestClassifier()
my_model2.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [88]:
y_pred = my_model2.predict(test_X)

In [89]:
# heads up accuracy
accuracy_score(test_y, y_pred)

0.099199999999999997

In [90]:
# F1 Score
print('Weighted: ', f1_score(test_y, y_pred, average='weighted'))
print('Macro: ', f1_score(test_y, y_pred, average='macro'))
print('Micro: ', f1_score(test_y, y_pred, average='micro'))

Weighted:  0.139004528481
Macro:  0.145979501323
Micro:  0.153560371517
